# Requires
Permit Status:
- Approved
- Refused
- Withdrawn
- Cancelled

Permit Description \
Permit Applicant Name \
Permit Number

In [158]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

In [160]:
url = 'https://www.shapeyourcity.ca/development'
PATH = "C:\Program Files (x86)\chromedriver-win64\chromedriver.exe"

In [184]:
driver = webdriver.Chrome(service = Service(PATH))
driver.get(url)
iframes = driver.find_elements(By.TAG_NAME, "iframe")
for iframe in iframes:
    driver.switch_to.frame(iframe)
    link = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.chakra-link.ehq-projectCoverImg.css-1eh7kaa')))
    urls = [item.get_attribute('href') for item in link]
    driver.switch_to.default_content()
urls

['https://www.shapeyourcity.ca/436-e-hastings-st',
 'https://www.shapeyourcity.ca/985-nicola-st',
 'https://www.shapeyourcity.ca/3581-w-30-ave-2',
 'https://www.shapeyourcity.ca/848-seymour-st-2',
 'https://www.shapeyourcity.ca/2775-e-hastings-st',
 'https://www.shapeyourcity.ca/1338-matthews-ave',
 'https://www.shapeyourcity.ca/1090-w-pender-st',
 'https://www.shapeyourcity.ca/385-lakewood-dr',
 'https://www.shapeyourcity.ca/3048-3080-w-broadway']